<a href="https://colab.research.google.com/github/Rifades/ELT-using-Google-Ecosystem/blob/main/(Personal)_Marico_Full_JBP_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import glob
from google.colab import drive
import re
from datetime import date
from pathlib import Path
import numpy as np

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path_billwise_itemwise = '/content/drive/My Drive/Marico JBP Work/Bill Wise Item Wise/January 2026'
folder_path_fillrate_lostreport = '/content/drive/My Drive/Marico JBP Work/Fill Rate Loss Report/January 2026'
folder_path_billprint = '/content/drive/My Drive/Marico JBP Work/Bill Print/January 2026'
folder_path_retailer_masterdata = '/content/drive/My Drive/Marico JBP Work/Retailer Master Data/January 2026'
folder_path_stockreport = '/content/drive/My Drive/Marico JBP Work/Stock Report/January 2026'

In [ ]:
file_list_billwise_itemwise = glob.glob(os.path.join(folder_path_billwise_itemwise, "*.xlsx")) + glob.glob(os.path.join(folder_path_billwise_itemwise, "*.xls"))
file_list_fillrate_lostreport = glob.glob(os.path.join(folder_path_fillrate_lostreport, "*.xlsx")) + glob.glob(os.path.join(folder_path_fillrate_lostreport, "*.xls"))
file_list_billprint = glob.glob(os.path.join(folder_path_billprint, "*.xlsx")) + glob.glob(os.path.join(folder_path_billprint, "*.xls"))
file_list_retailer_masterdata = glob.glob(os.path.join(folder_path_retailer_masterdata, "*.xlsx")) + glob.glob(os.path.join(folder_path_retailer_masterdata, "*.xls"))
file_list_stockreport = glob.glob(os.path.join(folder_path_stockreport, "*.xlsx")) + glob.glob(os.path.join(folder_path_stockreport, "*.xls"))

In [ ]:
dfs_billwise_itemwise = []
source_cols_billwise_itemwise = ['Bill Date', 'Bill Number', 'Status', 'Dsr Name','Beat Name', 'Retailer Name', 'ChannelName', 'NetAmount', 'Dist Code']

for file_path in file_list_billwise_itemwise:
    try:
        # Read the file without specifying usecols first to inspect actual columns
        temp_df = pd.read_excel(
            file_path,
            skiprows=[0, 1],
            header=0,
            sheet_name='Sheet1'
        )

        # Select only the required columns
        temp_df = temp_df[source_cols_billwise_itemwise]

        temp_df = temp_df[temp_df['Status'] == 'Delivered']
        temp_df['Bill Date'] = pd.to_datetime(temp_df['Bill Date']).dt.date
        dfs_billwise_itemwise.append(temp_df)

    except Exception as e:
        print(f"Error processing file '{file_path}': {e}. Skipping this file.")
        continue

dfs_billwise_itemwise = pd.concat(dfs_billwise_itemwise, ignore_index=True)
dfs_billwise_itemwise

,Bill Date,Bill Number,Status,Dsr Name,Beat Name,Retailer Name,ChannelName,NetAmount,Dist Code
0,2026-01-01,BILL250012057,Delivered,Mahafuzul Alam,FT_Mitha Dighi,Modina Store,Rural Cosmetics-RCS,507.000,16652
1,2026-01-01,BILL250012057,Delivered,Mahafuzul Alam,FT_Mitha Dighi,Modina Store,Rural Cosmetics-RCS,528.000,16652
2,2026-01-01,BILL250012080,Delivered,Mohammed Hossain,F1_Foltola,Jonota Store,Rural Grocer-RG,289.500,16652
3,2026-01-01,BILL250012080,Delivered,Mohammed Hossain,F1_Foltola,Jonota Store,Rural Grocer-RG,295.500,16652
4,2026-01-01,BILL250012080,Delivered,Mohammed Hossain,F1_Foltola,Jonota Store,Rural Grocer-RG,42.000,16652
...,...,...,...,...,...,...,...,...,...
85203,2026-01-15,BILL250044801,Delivered,Md. Firoz Shaik,F1_Shahpur,Tripty cosmetics,Rural Cosmetics-RCS,0.000,14686
85204,2026-01-15,BILL250044801,Delivered,Md. Firoz Shaik,F1_Shahpur,Tripty cosmetics,Rural Cosmetics-RCS,123.624,14686
85205,2026-01-15,BILL250044883,Delivered,Md. Robiul Islam,F1_Boikali,Urmi Store,Urban Neighborhood Grocer-UNG,404.496,14686
85206,2026-01-15,BILL250044841,Delivered,Md. Robayat Hossain,F1_Arong Ghata,VAI VAI ENT,Urban Cosmetics Store-UCS,528.000,14686


In [ ]:
dfs_fillrate_lostreport = []
source_cols_fillrate_lostreport = ['Salesman Name', 'Route Name', 'Retailer Name', 'Order Date','Sales Invoice No', 'Total Ordered Value', 'Total Delivered Value']

for file_path in file_list_fillrate_lostreport:
    try:
        # Read the file without specifying usecols first to inspect actual columns
        temp_df = pd.read_excel(
            file_path,
            skiprows=[0, 1],
            header=0,
            sheet_name='Sheet1'
        )

        # Select only the required columns
        temp_df = temp_df[source_cols_fillrate_lostreport]

        temp_df['Order Date'] = pd.to_datetime(temp_df['Order Date']).dt.date
        temp_df = temp_df.groupby(['Order Date', 'Sales Invoice No','Salesman Name', 'Route Name', 'Retailer Name'], as_index=False).agg({
            'Total Ordered Value': 'sum',
            'Total Delivered Value': 'sum'
        })
        dfs_fillrate_lostreport.append(temp_df)

    except Exception as e:
        print(f"Error processing file '{file_path}': {e}. Skipping this file.")
        continue

if dfs_fillrate_lostreport:
    dfs_fillrate_lostreport = pd.concat(dfs_fillrate_lostreport, ignore_index=True)
else:
    # Create an empty DataFrame with the expected columns if no files were processed
    columns_if_empty = ['Order Date', 'Sales Invoice No', 'Salesman Name', 'Route Name', 'Retailer Name', 'Total Ordered Value', 'Total Delivered Value']
    dfs_fillrate_lostreport = pd.DataFrame(columns=columns_if_empty)
    print("Warning: No fillrate lost report files were processed, resulting in an empty DataFrame.")

dfs_fillrate_lostreport

,Order Date,Sales Invoice No,Salesman Name,Route Name,Retailer Name,Total Ordered Value,Total Delivered Value
0,2026-01-01,BILL250032522,Hashem,FT_Kawel Jani,Bhai Bhai Cos,1209.60,437.68
1,2026-01-01,BILL250032523,Hashem,FT_Kawel Jani,Biskot Ghor,828.68,828.68
2,2026-01-01,BILL250032524,Hashem,FT_Kawel Jani,Bristi Cometic,3332.14,2865.74
3,2026-01-01,BILL250032525,Hashem,FT_Kawel Jani,Chowdhury store,1116.68,1116.68
4,2026-01-01,BILL250032526,Hashem,FT_Kawel Jani,Faijul G Store,3316.52,2850.12
...,...,...,...,...,...,...,...
44541,2026-01-17,BILL250037121,MD IQBAL,F1_BANDARTILA,Sifat Store,21.00,21.00
44542,2026-01-17,BILL250037122,MD IQBAL,F1_BANDARTILA,Sondip Store,575.28,575.28
44543,2026-01-17,BILL250037123,MD IQBAL,F1_BANDARTILA,Tin Bhai Store,21.00,21.00
44544,2026-01-17,BILL250037124,MD IQBAL,F1_BANDARTILA,Tin Bhai Store,3309.69,3309.69


In [ ]:
dfs_billprint = []
source_cols_billprint = ['SalInvno', 'NoofCopy', 'BilledUser']

for file_path in file_list_billprint:
    try:
        # Read the file without specifying usecols first to inspect actual columns
        temp_df = pd.read_excel(
            file_path,
            header=0,
            sheet_name='BillPrintUser'
        )

        # Select only the required columns
        temp_df = temp_df[source_cols_billprint]

        dfs_billprint.append(temp_df)

    except Exception as e:
        print(f"Error processing file '{file_path}': {e}. Skipping this file.")
        continue

dfs_billprint = pd.concat(dfs_billprint, ignore_index=True)
dfs_billprint['BillPrinted'] = 'Yes'
dfs_billprint

,SalInvno,NoofCopy,BilledUser,BillPrinted
0,BILL250069522,5,"MBL14862,MBL14862,MBL14862,MBL14862,MBL14862",Yes
1,BILL250069523,3,"MBL14862,MBL14862,MBL14862",Yes
2,BILL250069524,3,"MBL14862,MBL14862,MBL14862",Yes
3,BILL250069525,3,"MBL14862,MBL14862,MBL14862",Yes
4,BILL250069526,3,"MBL14862,MBL14862,MBL14862",Yes
...,...,...,...,...
44582,BILL250008816,1,MBL16601,Yes
44583,BILL250008817,1,MBL16601,Yes
44584,BILL250008818,1,MBL16601,Yes
44585,BILL250008819,1,MBL16601,Yes


In [ ]:
dfs_retailer_masterdata = []
source_cols_retailer_masterdata = ['Dist Code', 'Salesman Name', 'Route Name', 'Frequency','Retailer Name', 'Retailer Status', 'Frequency Type']

for file_path in file_list_retailer_masterdata:
    try:
        # Read the file without specifying usecols first to inspect actual columns
        temp_df = pd.read_excel(
            file_path,
            skiprows=[0, 1],
            header=0,
            sheet_name='Sheet1'
        )

        # Select only the required columns
        temp_df = temp_df[source_cols_retailer_masterdata]

        temp_df = temp_df[temp_df['Retailer Status'] == 'Active']
        dfs_retailer_masterdata.append(temp_df)

    except Exception as e:
        print(f"Error processing file '{file_path}': {e}. Skipping this file.")
        continue

dfs_retailer_masterdata = pd.concat(dfs_retailer_masterdata, ignore_index=True)
dfs_retailer_masterdata[dfs_retailer_masterdata['Route Name'] =='F1_GR_Amin Bazar 1']

,Dist Code,Salesman Name,Route Name,Frequency,Retailer Name,Retailer Status,Frequency Type
16347,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Mayer Dowa -2,Active,Weekly
16348,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Bismilla store,Active,Weekly
16349,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Soleman store,Active,Weekly
16350,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Bismillah - 2,Active,Weekly
16351,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Jamal cos,Active,Weekly
16352,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Polash cos,Active,Weekly
16353,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Sohid store Wholesals,Active,Weekly
16354,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Habib store,Active,Weekly
16355,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Shoukhin store,Active,Weekly
16356,14044,Amit Kumar Nandi,F1_GR_Amin Bazar 1,Weekly,Shad cos,Active,Weekly


In [ ]:
import pandas as pd
import re
from pathlib import Path
from datetime import date

# 1. Configuration
source_cols_stockreport = ['PrdDcode', 'PrdName', 'MRP', 'Saleable', 'DisplaySalRate']
dfs_stockreport = [] # Initialized once

for file_path in file_list_stockreport:
    path = Path(file_path)

    try:
        # 2. Read the Excel file
        temp_df = pd.read_excel(
            path,
            skiprows=[0, 1],
            header=0,
            usecols=source_cols_stockreport,
            sheet_name='Sheet1',
            engine='openpyxl'
        )

        # 3. Basic Cleaning (Remove TOTAL and empty product codes)
        temp_df = temp_df[temp_df['PrdDcode'].astype(str).str.upper() != 'TOTAL'].copy()
        temp_df = temp_df.dropna(subset=['PrdDcode'])

        # 4. Metadata extraction
        parts = path.stem.split('_', 1)
        if len(parts) == 2:
            dist_name = parts[1]
            file_id = parts[0]

            # Regex: (DD)(MM)(YYYY)
            match = re.search(r'(\d{2})(\d{2})(\d{4})', dist_name)

            if match:
                # --- FIXED DATE LOGIC ---
                month_val = int(match.group(1))   # Group 1 is Day
                day_val = int(match.group(2)) # Group 2 is Month
                year_val = int(match.group(3))  # Group 3 is Year

                # Assign metadata
                temp_df['Dist Name'] = dist_name
                temp_df['Dist Code'] = file_id
                # date() expects (year, month, day)
                temp_df['Date'] = pd.to_datetime(date(year_val, month_val, day_val))

                # 5. GroupBy Summary
                # Aggregating now reduces memory usage before appending to the list
                summary_df = temp_df.groupby(['Date', 'Dist Code'], as_index=False).agg({
                    'DisplaySalRate': 'sum'
                })

                dfs_stockreport.append(summary_df)

            else:
                print(f"⚠️ Date pattern not found in file_id: {file_id}")
        else:
            print(f"⚠️ Filename format error (missing underscore): {path.name}")

    except Exception as e:
        print(f"❌ Error processing {path.name}: {e}")

# 6. Final Consolidation
if dfs_stockreport:
    # Combine all small summaries into one final master summary
    dfs_stockreport = pd.concat(dfs_stockreport, ignore_index=True)

    # If there are multiple files for the same Dist/Date, sum them again
    dfs_stockreport = dfs_stockreport.groupby(['Date', 'Dist Code'], as_index=False).sum()

    print("✅ Summary Report Created Successfully.")
    print(dfs_stockreport)
else:
    print("🔍 No valid data processed.")

✅ Summary Report Created Successfully.
          Date      Dist Code  DisplaySalRate
0   2026-01-06  Copy of 12050    7.282194e+06
1   2026-01-06  Copy of 13438    4.308915e+06
2   2026-01-06  Copy of 14183    1.452450e+07
3   2026-01-06  Copy of 14686    4.106785e+06
4   2026-01-06  Copy of 14693    4.483719e+06
..         ...            ...             ...
136 2026-01-17  Copy of 15650    7.600917e+06
137 2026-01-17  Copy of 16548    3.389650e+06
138 2026-01-17  Copy of 16652    2.337770e+06
139 2026-01-17  Copy of 16653    4.635070e+06
140 2026-01-18  Copy of 14044    1.589591e+07

[141 rows x 3 columns]


In [ ]:
dfs_modified_billwise_itemwise = dfs_billwise_itemwise.groupby(['Bill Date', 'Bill Number', 'Status', 'Dsr Name', 'Beat Name', 'Retailer Name','ChannelName', 'Dist Code']).agg({'NetAmount': 'sum'}).reset_index()
dfs_modified_billwise_itemwise

,Bill Date,Bill Number,Status,Dsr Name,Beat Name,Retailer Name,ChannelName,Dist Code,NetAmount
0,1970-01-01,BILL250062013,Delivered,Nagen Dash,PSR (Lama Bazar Bualiya),Nagen Dash,PSR,13969,60561.60
1,2026-01-01,BILL250005117,Delivered,Md Zobed Ali,F2_Kalibari,Abir Store,Wholesale,16734,62192.67
2,2026-01-01,BILL250005118,Delivered,Md Zobed Ali,F2_Kalibari,Akondo Varities,Rural Cosmetics-RCS,16734,331.50
3,2026-01-01,BILL250005119,Delivered,Md Zobed Ali,F2_Kalibari,Aradha Cosmetics,Rural Cosmetics-RCS,16734,379.50
4,2026-01-01,BILL250005120,Delivered,Md Zobed Ali,F2_Kalibari,Arif store,Rural Grocer-RG,16734,331.50
...,...,...,...,...,...,...,...,...,...
38868,2026-01-16,BILL250102029,Delivered,Md Barkat,F1_Jamsin,Maliha store,Rural Grocer-RG,14044,21.00
38869,2026-01-16,BILL250102030,Delivered,Md Barkat,F1_Jamsin,Ma G store,Urban Neighborhood Grocer-UNG,14044,21.00
38870,2026-01-16,BILL250102031,Delivered,Md Barkat,F1_Jamsin,Allah Dan Store,Urban Cosmetics Store-UCS,14044,21.00
38871,2026-01-16,BILL250102032,Delivered,Md Barkat,F1_Jamsin,Alomgir Store,Urban Neighborhood Grocer-UNG,14044,7539.16


In [ ]:
# 1. Pre-Filtering & Cleaning Logic
def clean_and_filter(df, name_col, beat_col):
    """Removes PSR related data and ensures strings are standardized."""
    # Case-insensitive filter for 'psr' in both DSR and Beat names
    mask = (
        ~df[name_col].str.contains('psr', case=False, na=False) &
        ~df[beat_col].str.contains('psr', case=False, na=False)
    )
    df = df[mask].copy()

    # Standardize Dist Code to string to prevent '.0' suffixes or merge mismatches
    df['Dist Code'] = df['Dist Code'].astype(str).str.strip()
    return df

# Apply early filtering to both source dataframes
df_transaction_clean = clean_and_filter(dfs_modified_billwise_itemwise, 'Dsr Name', 'Beat Name')
df_master_clean = clean_and_filter(dfs_retailer_masterdata, 'Salesman Name', 'Route Name')

# 2. Aggregate Transaction (Served) Data
df_served = (
    df_transaction_clean
    .groupby(['Dist Code', 'Bill Date', 'Beat Name', 'Dsr Name'], as_index=False)
    .agg(served_retailers=('Retailer Name', 'nunique'))
)

# 3. Aggregate Master (Total) Data
df_master = (
    df_master_clean
    .groupby(['Dist Code', 'Route Name', 'Salesman Name'], as_index=False)
    .agg(total_retailers=('Retailer Name', 'nunique'))
)

# 4. Standardized Merge
# Note: left_on/right_on handles the column name differences (Beat vs Route)
df_final = pd.merge(
    df_served,
    df_master,
    left_on=['Dist Code', 'Beat Name', 'Dsr Name'],
    right_on=['Dist Code', 'Route Name', 'Salesman Name'],
    how='left'
)

# 5. Clean up Merge Keys and Handle Missing Master Data
df_final = df_final.drop(columns=['Salesman Name', 'Route Name'])
df_final['total_retailers'] = df_final['total_retailers'].fillna(0)

# 6. Optimized Calculation (Handling Division by Zero)
df_final['Served %'] = np.where(
    df_final['total_retailers'] > 0,
    (df_final['served_retailers'] / df_final['total_retailers']) * 100,
    0
)

# 7. Final Formatting
# Round the percentage and ensure columns are named cleanly
df_final['Served %'] = df_final['Served %'].round(2)
df_attendance_masterdata = df_final.sort_values(['Bill Date', 'Dist Code'])

# Display result
df_attendance_masterdata.head()

,Dist Code,Bill Date,Beat Name,Dsr Name,served_retailers,total_retailers,Served %
0,12050,2026-01-01,F1_GR_Charigram,Sohel Rana,23,50,46.00
1,12050,2026-01-01,F1_GR_Islamnagar-Pandhoa Bazar,Md. Anik Hossain,34,50,68.00
2,12050,2026-01-01,F2_GR_Shimulia-Sharifbag,MD Shahin Ali,27,51,52.94
3,12050,2026-01-01,F2_Joypura-Jalsin,Sharif Deoyan,22,46,47.83
4,12050,2026-01-01,F2_Kalampur Bazar,Md. Mokbul Hossain,26,51,50.98


In [ ]:
df_Billprint_report = pd.merge(
    dfs_modified_billwise_itemwise,
    dfs_billprint,
    left_on=['Bill Number'],
    right_on=['SalInvno'],
    how='left'
)
df_Billprint_report['BillPrinted'] = df_Billprint_report['BillPrinted'].fillna('No')

df_Billprint_report = df_Billprint_report.drop(columns=['BilledUser', 'NoofCopy', 'NetAmount', 'Status', 'ChannelName', 'NetAmount', 'SalInvno', 'NoofCopy', 'BilledUser'])
df_Billprint_report

,Bill Date,Bill Number,Dsr Name,Beat Name,Retailer Name,Dist Code,BillPrinted
0,1970-01-01,BILL250062013,Nagen Dash,PSR (Lama Bazar Bualiya),Nagen Dash,13969,Yes
1,2026-01-01,BILL250005117,Md Zobed Ali,F2_Kalibari,Abir Store,16734,Yes
2,2026-01-01,BILL250005118,Md Zobed Ali,F2_Kalibari,Akondo Varities,16734,Yes
3,2026-01-01,BILL250005119,Md Zobed Ali,F2_Kalibari,Aradha Cosmetics,16734,Yes
4,2026-01-01,BILL250005120,Md Zobed Ali,F2_Kalibari,Arif store,16734,Yes
...,...,...,...,...,...,...,...
66427,2026-01-16,BILL250102029,Md Barkat,F1_Jamsin,Maliha store,14044,Yes
66428,2026-01-16,BILL250102030,Md Barkat,F1_Jamsin,Ma G store,14044,Yes
66429,2026-01-16,BILL250102031,Md Barkat,F1_Jamsin,Allah Dan Store,14044,Yes
66430,2026-01-16,BILL250102032,Md Barkat,F1_Jamsin,Alomgir Store,14044,Yes


In [ ]:
print(dfs_fillrate_lostreport.columns)
print(dfs_modified_billwise_itemwise.columns)

Index(['Order Date', 'Sales Invoice No', 'Salesman Name', 'Route Name',
       'Retailer Name', 'Total Ordered Value', 'Total Delivered Value'],
      dtype='object')
Index(['Bill Date', 'Bill Number', 'Status', 'Dsr Name', 'Beat Name',
       'Retailer Name', 'ChannelName', 'Dist Code', 'NetAmount'],
      dtype='object')


In [ ]:
df_ordervsdelivery_report = pd.merge(
    dfs_fillrate_lostreport,
    dfs_modified_billwise_itemwise,
    left_on=['Sales Invoice No', 'Salesman Name', 'Route Name', 'Retailer Name'],
    right_on=['Bill Number', 'Dsr Name', 'Beat Name', 'Retailer Name'],
    how='left'
)
df_ordervsdelivery_report = df_ordervsdelivery_report.drop(columns=['Bill Date', 'Status', 'NetAmount', 'Status', 'Dsr Name', 'NetAmount', 'Beat Name','ChannelName'])
# Drops only the rows where 'Bill Number' is NaN
df_ordervsdelivery_report = df_ordervsdelivery_report.dropna(subset=['Bill Number'])
df_ordervsdelivery_report

,Order Date,Sales Invoice No,Salesman Name,Route Name,Retailer Name,Total Ordered Value,Total Delivered Value,Bill Number,Dist Code
0,2026-01-01,BILL250032522,Hashem,FT_Kawel Jani,Bhai Bhai Cos,1209.60,437.68,BILL250032522,14693.0
1,2026-01-01,BILL250032523,Hashem,FT_Kawel Jani,Biskot Ghor,828.68,828.68,BILL250032523,14693.0
2,2026-01-01,BILL250032524,Hashem,FT_Kawel Jani,Bristi Cometic,3332.14,2865.74,BILL250032524,14693.0
3,2026-01-01,BILL250032525,Hashem,FT_Kawel Jani,Chowdhury store,1116.68,1116.68,BILL250032525,14693.0
4,2026-01-01,BILL250032526,Hashem,FT_Kawel Jani,Faijul G Store,3316.52,2850.12,BILL250032526,14693.0
...,...,...,...,...,...,...,...,...,...
44219,2026-01-15,BILL250036799,MD IQBAL,F1_GR_TALTOLA,Shoriyat Pur Store,650.08,650.08,BILL250036799,16653.0
44220,2026-01-15,BILL250036800,MD IQBAL,F1_GR_TALTOLA,Subarna Store,3452.37,3452.37,BILL250036800,16653.0
44221,2026-01-15,BILL250036801,MD IQBAL,F1_GR_TALTOLA,Tarek store,3553.05,3553.05,BILL250036801,16653.0
44222,2026-01-15,BILL250036802,MD IQBAL,F1_GR_TALTOLA,Vhai Bon Store,487.40,487.40,BILL250036802,16653.0


In [ ]:
df_ordervsdelivery_report[df_ordervsdelivery_report['Bill Number'] == 'BILL250041684' ]

,Order Date,Sales Invoice No,Salesman Name,Route Name,Retailer Name,Total Ordered Value,Total Delivered Value,Bill Number,Dist Code
29395,2026-01-10,BILL250041684,Sharif Deoyan,F2_GR_Bathuli-Rajnagar,Ma Varaitics Store (2),554.28,554.28,BILL250041684,12050.0
30425,2026-01-01,BILL250041684,Md Nijam Uddin,F1_Amua Bazar,Abul Salam store,954.27,954.27,BILL250041684,13438.0


In [ ]:
dfs_stocknormsreport = dfs_stockreport.groupby(['Dist Code']).agg({'DisplaySalRate': 'mean'}).reset_index()

dfs_stocknormsreport['Dist Code'] = dfs_stocknormsreport['Dist Code'].astype(str).str.extract(r'(\d+)').astype(int)
dfs_stocknormsreport

,Dist Code,DisplaySalRate
0,12050,5.820615e+06
1,13438,3.521568e+06
2,13948,7.217107e+06
3,13969,1.303200e+07
4,14044,1.981360e+07
5,14183,9.311490e+06
6,14686,3.530313e+06
7,14693,4.015886e+06
8,14862,1.047940e+07
9,15650,7.996362e+06


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import time

# 1. Define the target Spreadsheet and Tab
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1zxw_7Yo8EOAr9CJgIbb5daDLjDZSAO6hohBZcaC7kYo/edit?gid=0#gid=0'
sheet_name = 'Attendance_Master_Data' # The name of the tab inside the file

# 2. Open the spreadsheet
sh = gc.open_by_url(spreadsheet_url)

# 3. Open or Create the specific worksheet
try:
    worksheet = sh.worksheet(sheet_name)
except gspread.exceptions.WorksheetNotFound:
    # If the tab doesn't exist, create it
    worksheet = sh.add_worksheet(title=sheet_name, rows="10000", cols="20")

# 4. Clear the existing content before writing new data
# Adding a delay to mitigate Google Sheets API quota limits
time.sleep(5) # Wait for 5 seconds before clearing
worksheet.clear()

# 5. Export the DataFrame
# include_index=False prevents the row numbers (0, 1, 2...) from being uploaded
set_with_dataframe(worksheet, df_attendance_masterdata, include_index=False)

print(f"✅ Data exported successfully to {sheet_name}!")

✅ Data exported successfully to Attendance_Master_Data!


In [ ]:
# 1. Map your Sheet Names to your DataFrames
my_reports = {
    'Attendance_Master_Data': df_attendance_masterdata,
    'Stock_Summary': dfs_stocknormsreport,
    'Bill_Print_Analysis': df_Billprint_report,
    'Order_vs_Delivery_Analysis': df_ordervsdelivery_report
}

# 2. Define your URL
target_url = 'https://docs.google.com/spreadsheets/d/1zxw_7Yo8EOAr9CJgIbb5daDLjDZSAO6hohBZcaC7kYo/edit?gid=0#gid=0'


In [ ]:
from gspread_dataframe import set_with_dataframe
import gspread

def export_multiple_dfs_to_google_sheet(spreadsheet_url, data_dict):
    """
    Exports large DataFrames by first shrinking the sheet to the exact
    required dimensions to stay under the 10-million cell limit.
    """
    try:
        sh = gc.open_by_url(spreadsheet_url)
    except Exception as e:
        print(f"❌ Could not open spreadsheet: {e}")
        return

    for sheet_name, df in data_dict.items():
        try:
            # 1. Get or Create Worksheet
            try:
                worksheet = sh.worksheet(sheet_name)
            except gspread.exceptions.WorksheetNotFound:
                worksheet = sh.add_worksheet(title=sheet_name, rows="1", cols="1")

            # 2. THE CRITICAL STEP: Resize to exact DF dimensions
            # Rows = len(df) + 1 (for header), Cols = number of columns
            rows_needed = len(df) + 1
            cols_needed = len(df.columns)

            print(f"🔄 Resizing {sheet_name} to {rows_needed}x{cols_needed}...")
            worksheet.resize(rows=rows_needed, cols=cols_needed)

            # 3. Clear and Write
            worksheet.clear()
            set_with_dataframe(worksheet, df, include_index=False)
            print(f"✅ Successfully updated: {sheet_name}")

        except Exception as e:
            print(f"❌ Failed to update {sheet_name}: {e}")

    print("\n🚀 All exports completed!")

In [ ]:
export_multiple_dfs_to_google_sheet(target_url, my_reports)

🔄 Resizing Attendance_Master_Data to 1210x7...
✅ Successfully updated: Attendance_Master_Data
🔄 Resizing Stock_Summary to 16x2...
✅ Successfully updated: Stock_Summary
🔄 Resizing Bill_Print_Analysis to 66433x7...
✅ Successfully updated: Bill_Print_Analysis
🔄 Resizing Order_vs_Delivery_Analysis to 35427x9...
✅ Successfully updated: Order_vs_Delivery_Analysis

🚀 All exports completed!


In [ ]:
!pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
st.title('Hello')
st.line_chart(dfs_stocknormsreport)

Overwriting app.py


In [ ]:
# 1. Get the public IP address (you will need to copy this)
!wget -q -O - ipv4.icanhazip.com

# 2. Run Streamlit in the background & expose it via localtunnel
!streamlit run app.py & npx localtunnel --port 8501

34.150.196.85
⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴your url is: https://slick-owls-learn.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.150.196.85:8501

────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.12/dist-packages/streamlit/runtime/scriptrunner/exec_code.py:  
  129 in exec_func_with_error_handling                                                  
                                                                                        
  /usr/local/lib/python3.12/dist-packages/streamlit/runtime/scriptrunner/script_runner  
  .py:672 in code_to_exec                                                               
                                                                                        
  /content/app.py:3 in <module>                                                         
                                     